# Clasificador bayesiano ingenuo de Bernoulli
En esta libreta programaremos un clasificador bayesiano ingenuo en el cual se presupone que la distribución de los atributos dada la clase es una Bernoulli y hay dos posibles clases. 

## Tarea 1 Aprendizaje automatizado
## María Carmen Aguirre Delgado

In [1]:
import pandas as pd
import random
import math
import numpy as np 
from sklearn.model_selection import train_test_split

## Carga y lectura de datos 
#### El archivo spam.csv contiene 2001 valores por cada renglón, de los cuales los primeros 2000 representan el histograma de palabras de un correo y el último corresponde a la clase, esto es, 1 si es spam y 0 si no lo es.

In [2]:
url = 'http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv'
df = pd.read_csv(url, sep=' ', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 1. Reporta el porcentaje de correos que están etiquetados como spam y como no spam en el conjunto de datos

In [3]:
#rename the last column 
df.rename(columns = {2000 : 'SPAM'}, inplace=True) 

spam = len(df[df['SPAM'] == 1])
no_spam = len(df[df['SPAM'] == 0])

print(f'La cantidad de correos marcados como spam es {spam}')
print(f'Lo cual representa un porcentaje del {spam*100/(spam + no_spam)} %')
print(f'La cantidad de correos no marcados como spam es {no_spam}')
print(f'Lo cual representa un porcentaje del {no_spam*100/(spam + no_spam)} %')

La cantidad de correos marcados como spam es 1500
Lo cual representa un porcentaje del 29.00232018561485 %
La cantidad de correos no marcados como spam es 3672
Lo cual representa un porcentaje del 70.99767981438515 %


### 2. Divide aleatoriamente el conjunto de datos en el 60 % para entrenamiento, el 20 % para validación y el 20 % restante para prueba usando 22 como semilla para tu generador de números aleatorios

In [4]:
X = df.loc[:, df.columns != 'SPAM']
y =df['SPAM']

  
# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                          test_size = 0.2,
                          random_state = 22,
                          train_size = 0.6, shuffle = True)
  

In [5]:
# get the validation set
tmp = df[ ~df.index.isin(y_train.index) ]
tmp2 = tmp[ ~tmp.index.isin(y_test.index) ]

X_validation = tmp2.loc[:, tmp2.columns != 'SPAM']
Y_validation = tmp2['SPAM']

In [6]:
print(f'El conjunto de entrenamiento tiene {len(X_train)} observaciones.')
print(f'El conjunto de validación tiene {len(X_test)} observaciones.')
print(f'El conjunto de prueba tiene {len(X_validation)} observaciones.')

El conjunto de entrenamiento tiene 3103 observaciones.
El conjunto de validación tiene 1035 observaciones.
El conjunto de prueba tiene 1034 observaciones.


## Entrena 2 clasificadores bayesianos ingenuos con distintas distribuciones.

### Clasificador bayesiano ingenuo para distribución de Bernoulli
#### Definimos una función para obtener la probabilidad de 0 o 1 dada una distribución de Bernoulli con parámetro $q$. Definimos una clase con el clasificador bayesiano ingenuo para atributos binarios y estimación de parámetros por máxima verosimilitud.


In [7]:
class NaiveBayesClassifier:
    
    def __init__(self, estimator = 'maxLikelihood', alpha = 2):
        estimators = ['maxLikelihood', 'maxAPosteriori']
        self.alpha = alpha
        # Validamos que el estimador seleccionado sea correcto
        if estimator not in estimators:
            print('Estimators not found. Try maxLikelihood or maxAPosteriori.')
        else:
            self.estimator = estimator

    # Estimación de parámetros por máxima verosimilitud
    def maxLikelihoodEstimation(self):
        pass

    # Estimación de parámetros por máxima a posteriori
    def maxAPosterioriEstimation(self):
        pass

    # Ecuación de verosimilitud para una distribución de Bernoulli
    def distributionFunction(self, X):
        pass

    # Entrenamos el modelo
    def fit(self, X, Y):
        self.X_train = X
        self.Y_train = Y
        self.n = X_train.shape[0] # Cantidad de observaciones
        self.n_atrib = X_train.shape[1] # Cantidad de parámetros
        self.classes = np.unique(self.Y_train) 
        self.n_classes = self.classes.size # Cantidad de clases
        # Estimamos los parámetros para las clases y los atributos
        self.qc = np.zeros(self.classes.size) 
        self.qa = np.zeros((self.n_classes, self.n_atrib))
        if self.estimator == 'maxLikelihood':
            self.maxLikelihoodEstimation()
        else:
            self.maxAPosterioriEstimation()
        self.applyLn()

    # Aplixamos el logaritmo a los parámetros para evitar desborde de datos
    def applyLn(self):
        self.qc[self.qc == 0] = np.nextafter(0, 1)
        self.qc[self.qc == 1] = np.nextafter(1, 0)
        self.qc = np.log(self.qc)
        self.qa[self.qa == 0] = np.nextafter(0, 1)
        self.qa[self.qa == 1] = np.nextafter(1, 0)
        self.qa = np.log(self.qa)

    # Clasificamos nuevas observaciones de acuerdo al modelo entrenado
    def predict(self, X):
        X_test = X
        Y_test = np.zeros(X_test.shape[0])
        # Obtenemos las verosimilitudes del conjunto de datos de prueba
        Likelihood = self.qc + self.distributionFunction(X_test)
        # Clasificando de acuerdo al valor máximo de verosimilitud
        for i, L in enumerate(Likelihood):
            Y_test[i] = self.classes[np.argmax(L)]
        return Y_test



In [8]:
class NBCbernoulli(NaiveBayesClassifier):
    # Estimación de parámetros por máxima verosimilitud
    def maxLikelihoodEstimation(self):
        for i, c in enumerate(self.classes):
            Xc = self.X_train[np.where(self.Y_train == c)]
            nc = Xc.shape[0]
            self.qc[i] = nc / self.n
            self.qa[i] =  Xc.sum(axis = 0) / nc

    # Estimación de parámetros por máxima a posteriori
    def maxAPosterioriEstimation(self):
        for i, c in enumerate(self.classes):
            Xc = self.X_train[np.where(self.Y_train == c)]
            nc = Xc.shape[0]
            self.qc[i] = (nc + self.alpha - 1) / (self.n + self.n_atrib + self.alpha - 2)
            self.qa[i] =  (Xc.sum(axis = 0) + self.alpha - 1) / (nc + self.n_atrib + self.alpha - 2)

    # Ecuación de verosimilitud para una distribución de Bernoulli
    def bernoulli(self, x, q):
        """
        Distribución de bernoulli
        """
        return ((1 - x) @ (1 - q)) + (x @ (q))

    def distributionFunction(self, X):
        L = np.zeros((X.shape[0], self.classes.size))
        for i in range(self.n_classes):
            L[:, i] = self.qc[i] + self.bernoulli(X, self.qa[i, :])
        return L

In [9]:
class NBCcategorical(NaiveBayesClassifier):
    # Estimación de parámetros por máxima verosimilitud
    def maxLikelihoodEstimation(self):
        self.n_cat = int(np.max(self.X_train)) + 1
        self.qa = np.zeros((self.n_classes, self.n_atrib, self.n_cat))
        for i, c in enumerate(self.classes):
            Xc = self.X_train[np.where(self.Y_train == c)]
            nc = Xc.shape[0]
            self.qc[i] = nc / self.n_atrib
            for j in range(self.n_atrib):
                Xci = self.X_train[np.where(self.Y_train == c), j]
                for k in range(self.n_cat): 
                    self.qa[i, j, k] = np.sum(Xci == k)  / Xci.shape[0]

    # Estimación de parámetros por máxima a posteriori
    def maxAPosterioriEstimation(self):
        self.n_cat = int(np.max(self.X_train)) + 1
        self.qa = np.zeros((self.n_classes, self.n_atrib, self.n_cat))
        for i, c in enumerate(self.classes):
            Xc = self.X_train[np.where(self.Y_train == c)]
            nc = Xc.shape[0]
            self.qc[i] = (np.sum(self.Y_train == c) + self.alpha - 1) / (self.n + self.alpha * self.n_classes - self.n_classes)
            for j in range(self.n_atrib):
                Xci = self.X_train[np.where(self.Y_train == c), j]
                for k in range(self.n_cat): 
                    self.qa[i, j, k] = (np.sum(Xci == k) + self.alpha - 1) / (Xci.shape[0] + self.alpha * self.n_cat - self.n_cat)

    def categorical(self, c, q):
      """
      Distribución categorica
      """
      return q[int(c)]
    
    # Ecuación de verosimilitud para una distribución de Categórica
    def distributionFunction(self, X):
        L = np.zeros((X.shape[0], self.classes.size))
        for k in range(X.shape[0]):
            for i in range(self.classes.size):
                for j in range(self.n_atrib):
                    L[k, i] = L[k, i] + self.categorical(X[k, j], self.qa[i, j, :])
        return L
    

## Train

In [10]:
#X_train, X_test, y_train, y_test
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
Y_train = np.asarray(y_train)
Y_test = np.asarray(y_test)

NBC_Bernoulli = NBCbernoulli('maxLikelihood')
NBC_Bernoulli.fit(X_train, Y_train)

NBC_Categorical = NBCcategorical('maxLikelihood')
NBC_Categorical.fit(X_train, Y_train)

NBC_Bernoulli2 = NBCbernoulli('maxAPosteriori')
NBC_Bernoulli2.fit(X_train, Y_train)

NBC_Categorical2 = NBCcategorical('maxAPosteriori')
NBC_Categorical2.fit(X_train, Y_train)



## Evaluation

In [11]:
y_bnb_test = NBC_Bernoulli.predict(X_test)
evaluation_bnb_test = np.mean(y_bnb_test == Y_test)

y_bnb_train = NBC_Bernoulli.predict(X_train)
evaluation_bnb_train = np.mean(y_bnb_train == Y_train)

y_cat_test = NBC_Categorical.predict(X_test)
evaluation_cat_test = np.mean(y_cat_test == Y_test)

y_cat_train = NBC_Categorical.predict(X_train)
evaluation_cat_train = np.mean(y_cat_train == Y_train)

y_bnb_test2 = NBC_Bernoulli2.predict(X_test)
evaluation_bnb_test2 = np.mean(y_bnb_test2 == Y_test)

y_bnb_train2 = NBC_Bernoulli2.predict(X_train)
evaluation_bnb_train2 = np.mean(y_bnb_train2 == Y_train)

y_cat_test2 = NBC_Categorical2.predict(X_test)
evaluation_cat_test2 = np.mean(y_cat_test2 == Y_test)

y_cat_train2 = NBC_Categorical2.predict(X_train)
evaluation_cat_train2 = np.mean(y_cat_train2 == Y_train)

In [12]:
# Evaluamos el modelo
print("Evaluacion de BNB con maxLikelihood")
print(f'Para el conjunto de entrenamiento: {evaluation_bnb_train*100:.2f}%')
print(f'Para el conjunto de prueba: {evaluation_bnb_test*100:.2f}%')

print("\nEvaluacion de Cat con maxLikelihood")
print(f'Para el conjunto de entrenamiento: {evaluation_cat_train*100:.2f}%')
print(f'Para el conjunto de prueba: {evaluation_cat_test*100:.2f}%')

print("\nEvaluacion de BNB con maxAPosteriori")
print(f'Para el conjunto de entrenamiento: {evaluation_bnb_train2*100:.2f}%')
print(f'Para el conjunto de prueba: {evaluation_bnb_test2*100:.2f}%')

print("\nEvaluacion de Cat con maxAPosteriori")
print(f'Para el conjunto de entrenamiento: {evaluation_cat_train2*100:.2f}%')
print(f'Para el conjunto de prueba: {evaluation_cat_test2*100:.2f}%')

Evaluacion de BNB con maxLikelihood
Para el conjunto de entrenamiento: 70.74%
Para el conjunto de prueba: 71.21%

Evaluacion de Cat con maxLikelihood
Para el conjunto de entrenamiento: 86.92%
Para el conjunto de prueba: 85.51%

Evaluacion de BNB con maxAPosteriori
Para el conjunto de entrenamiento: 70.80%
Para el conjunto de prueba: 71.30%

Evaluacion de Cat con maxAPosteriori
Para el conjunto de entrenamiento: 70.61%
Para el conjunto de prueba: 71.21%


# Using sklearn

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [14]:
def test_classifier(classifier, x_train, y_train, x_test, y_test):
  model = classifier
  model.fit(x_train, y_train)

  y_train_m = np.mean(y_train == model.predict(x_train))
  y_test_m = np.mean(y_test == model.predict(x_test))
  
  y_pred = model.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)

  # print de resultados
  print('\nPerformance para: ', classifier)
  print(f'Para el conjunto de entrenamiento: {y_train_m*100:.2f}%')
  print(f'Para el conjunto de prueba: {y_test_m*100:.2f}%')
  print("Accuracy_score: ", accuracy)
 # return y_train, y_test

In [15]:
to_test =[MultinomialNB(), GaussianNB(),  BernoulliNB()]
for i in range(len(to_test)):
  classifier = to_test[i]
  test_classifier(classifier, X_train, Y_train, X_test, Y_test)


Performance para:  MultinomialNB()
Para el conjunto de entrenamiento: 95.68%
Para el conjunto de prueba: 94.59%
Accuracy_score:  0.9458937198067633

Performance para:  GaussianNB()
Para el conjunto de entrenamiento: 93.68%
Para el conjunto de prueba: 92.95%
Accuracy_score:  0.9294685990338164

Performance para:  BernoulliNB()
Para el conjunto de entrenamiento: 89.95%
Para el conjunto de prueba: 90.14%
Accuracy_score:  0.9014492753623189


# Resultados
Los mejores resultados se obtuvieron para nuestro clasificador \
Evaluacion de Cat con maxLikelihood\
Para el conjunto de entrenamiento: 86.92%\
Para el conjunto de prueba: 85.51%\
\
Por otro lado, al usar los modelos de sklearn el rendimiento mejora significativamente al usar un clasificador bayesiano con distribución mutinomial:\
\
Performance para:  MultinomialNB()\
Para el conjunto de entrenamiento: 95.68%\
Para el conjunto de prueba: 94.59%\
Accuracy_score:  0.9458937198067633

# Conjunto de validación

In [16]:
X_validation = np.asarray(X_validation)
Y_validation = np.asarray(Y_validation)

In [17]:
y_cat_val = NBC_Categorical.predict(X_validation)
evaluation_cat_validation = np.mean(y_cat_val == Y_validation)

print("\nEvaluacion de NB Categorico con maxLikelihood")
print(f'Para el conjunto de validation: {evaluation_cat_validation*100:.2f}%')


Evaluacion de NB Categorico con maxLikelihood
Para el conjunto de validation: 86.17%


In [18]:
print("Usando el clasificador de sklear para los datos de validacion")
test_classifier(MultinomialNB(), X_train, Y_train, X_validation, Y_validation)

Usando el clasificador de sklear para los datos de validacion

Performance para:  MultinomialNB()
Para el conjunto de entrenamiento: 95.68%
Para el conjunto de prueba: 95.16%
Accuracy_score:  0.9516441005802708


Vemos que el mejor performance se obtine usando sklearn, para una distribución MultinomialNB, por otro lado para nuestros modelos los mejores resultados son para un clasificador NB con distribución categorica y usando maxLikelihood como parámetro. 